In [1]:
import pandas as pd

# Cargar los datos de los campeones desde el archivo JSON
df_campeones = pd.read_json('estadisticasCampeones.json')

# Visualizar los primeros registros para confirmar la carga
df_campeones.head()
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Extraer los IDs de los campeones para el one-hot encoding
champion_ids = df_campeones['Key'].values.reshape(-1, 1)

# Crear el encoder y transformar los datos
encoder = OneHotEncoder(sparse=False)
champion_onehot = encoder.fit_transform(champion_ids)

# Normalizar las estadísticas
scaler = MinMaxScaler()
stats_scaled = scaler.fit_transform(df_campeones[['Difficulty', 'HP', 'Armor', 'HPRegen', 'AttackDamage', 'AttackSpeed']])

# Combinar one-hot encoded IDs con estadísticas normalizadas
import numpy as np
champion_features = np.hstack((champion_onehot, stats_scaled))

# Mostrar las dimensiones del conjunto de características resultante
champion_features.shape


,ID,Key,Name,Difficulty,HP,Armor,HPRegen,AttackDamage,AttackSpeed
0,Aatrox,266,Aatrox,4,650,38,3.00,60,0.651
1,Ahri,103,Ahri,5,590,21,2.50,53,0.668
2,Akali,84,Akali,7,570,23,9.00,62,0.625
3,Akshan,166,Akshan,0,630,26,3.75,52,0.638
4,Alistar,12,Alistar,7,685,47,8.50,62,0.625


In [2]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Extraer los IDs de los campeones para el one-hot encoding
champion_ids = df_campeones['Key'].values.reshape(-1, 1)

# Crear el encoder y transformar los datos
encoder = OneHotEncoder(sparse=False)
champion_onehot = encoder.fit_transform(champion_ids)

# Normalizar las estadísticas
scaler = MinMaxScaler()
stats_scaled = scaler.fit_transform(df_campeones[['Difficulty', 'HP', 'Armor', 'HPRegen', 'AttackDamage', 'AttackSpeed']])

# Combinar one-hot encoded IDs con estadísticas normalizadas
import numpy as np
champion_features = np.hstack((champion_onehot, stats_scaled))

# Mostrar las dimensiones del conjunto de características resultante
champion_features.shape


C:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(164, 170)

In [4]:
import json
import numpy as np

with open('arraysPartidasResultado.json', 'r') as file:
    partidas_data = json.load(file)

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
champion_onehot = encoder.fit_transform(champion_ids)
# Extraer características y etiquetas
partidas_features = []
partidas_labels = []

for partida in partidas_data:
    
    partidas_labels.append(partida[0])
   
    champions = partida[1:]
    #Convertimos a one-hot
    one_hot_encoded = encoder.transform(np.array(champions).reshape(-1, 1)).flatten()
    partidas_features.append(one_hot_encoded)


# Convertir listas a arrays de NumPy

partidas_features = np.array(partidas_features)
partidas_labels = np.array(partidas_labels)

# Visualizar dimensiones para confirmar
partidas_features.shape, partidas_labels.shape


C:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


((13783, 1640), (13783,))

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Crear el modelo con regularización y dropout
model = Sequential([
    Input(shape=(1640,)),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
history = model.fit(partidas_features, partidas_labels, epochs=10, batch_size=32, validation_split=0.2)



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │       210,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 220,417 (861.00 KB)

 Trainable params: 220,417 (861.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5338 - loss: 1.7872 - val_accuracy: 0.5502 - val_loss: 0.6953
Epoch 2/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5383 - loss: 0.6931 - val_accuracy: 0.5502 - val_loss: 0.6883
Epoch 3/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5471 - loss: 0.6887 - val_accuracy: 0.5502 - val_loss: 0.6882
Epoch 4/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5424 - loss: 0.6901 - val_accuracy: 0.5502 - val_loss: 0.6881
Epoch 5/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5516 - loss: 0.6880 - val_accuracy: 0.5502 - val_loss: 0.6886
Epoch 6/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5487 - loss: 0.6889 - val_accuracy: 0.5502 - val_loss: 0.6886
Epoch 7/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5486 - loss: 0.6890 - val_accuracy: 0.5502 - val_loss: 0.6883
Epoch 8/10
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5431 - loss: 0.6894 - val_accuracy: 0.

     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/377.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/377.0 MB 7.1 MB/s eta 0:00:53
   ---------------------------------------- 1.3/377.0 MB 13.9 MB/s eta 0:00:28
    --------------------------------------- 4.9/377.0 MB 34.7 MB/s eta 0:00:11
    --------------------------------------- 7.2/377.0 MB 46.1 MB/s eta 0:00:09
    --------------------------------------- 7.9/377.0 MB 36.3 MB/s eta 0:00:11
   - -------------------------------------- 9.6/377.0 MB 38.3 MB/s eta 0:00:10
   - -------------------------------------- 12.4/377.0 MB 46.7 MB/s eta 0:00:08
   - -------------------------------------- 14.3/377.0 MB 40.9 MB/s eta 0:00:09
   - -------------------------------------- 15.9/377.0 MB 36.4 MB/s eta 0:00:10
   -- ------------------------------------- 19.8/377.0 MB 50.4 M